# Table Question generation: HybridQA dataset
In this notebook, we will see how to fine-tune and evaluate a question generation model on HybridQA dataset.

## Configuration

We start by setting some parameters to configure the process.  Note that depending on the GPU being used you may need to tune the batch size.

In [1]:
model_name_or_path="t5-small"
modality="hybrid"
dataset_name="hybrid_qa"
max_len=200
target_max_len=40
output_dir="/dccstor/cssblr/rbhat//models/"
learning_rate=0.0001
num_train_epochs=2
per_device_train_batch_size=8
per_device_eval_batch_size=32
evaluation_strategy='epoch'

In [2]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    evaluation_strategy='epoch',
    learning_rate=learning_rate,
    prediction_loss_only=True,
    remove_unused_columns=False,
    )
training_args.predict_with_generate=True
training_args.remove_unused_columns = False
training_args.prediction_loss_only = False

---
## HybridQA data
Here we load one instance of HybridQA and visualize it. <font color='red'>This part of the code is not needed to train the model </font>

In [3]:
import json
from datasets import load_dataset

def print_hybridqa_instance(train_instance):
    print(json.dumps(train_instance, indent=4))

train_instance = load_dataset('hybrid_qa', split='train[1001:1002]')[0]
print_hybridqa_instance(train_instance)

Reusing dataset hybrid_qa (/u/rbhat/.cache/huggingface/datasets/hybrid_qa/hybrid_qa/1.0.0/fabdc38783449dd6cb1acd25621af97b871e218fc3ab608191d492b408a93ab8)


{
    "question_id": "0424073b0d76fcb3",
    "question": "The tracks of what creature are found in the formation located in the largest country in Southern Europe ?",
    "table_id": "List_of_stratigraphic_units_with_ornithischian_tracks_5",
    "answer_text": "Pterosaur",
    "question_postag": "DT NNS IN WP NN VBP VBN IN DT NN VBN IN DT JJS NN IN NNP NNP .",
    "table": {
        "url": "https://en.wikipedia.org/wiki/List_of_stratigraphic_units_with_ornithischian_tracks",
        "title": "List of stratigraphic units with ornithischian tracks",
        "header": [
            "Name",
            "Location",
            "Description"
        ],
        "data": [
            {
                "value": "Aganane Formation",
                "urls": [
                    {
                        "url": "/wiki/Aganane_Formation",
                        "summary": "The Aganane Formation is a Pliensbachian geologic formation in Morocco . Fossil stegosaur and theropod tracks have been repor

The SQL gets converted to a string format which goes as input to generator to generate question

In [4]:
from primeqa.qg.processors.hybrid_qg.hybridqa_processor import HybridQADataset

data = HybridQADataset()
processed_data = data.preprocess_data_for_qg('train[1001:1002]')
print('Question = ', processed_data['question'][0])
print('\nInput to generator = ', processed_data['input'][0])

Reusing dataset hybrid_qa (/u/rbhat/.cache/huggingface/datasets/hybrid_qa/hybrid_qa/1.0.0/fabdc38783449dd6cb1acd25621af97b871e218fc3ab608191d492b408a93ab8)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
Extracting chains: 1it [00:00, 358.06it/s]

Question =  The tracks of what creature are found in the formation located in the largest country in Southern Europe ?

Input to generator =  Pterosaur  <<answer>>  The Lastres Formation is a geological formation located in Asturias province , northwestern Spain Fossil pterosaur tracks have been found in the formation . <<hsep>> The Name is Lastres Formation. <<hsep>> The Location is Spain. <<header>> List of stratigraphic units with ornithischian tracks <<header>> Thyreophorans -- Stegosaurs


---
## Loading the Model

Here we load the model based on the model_name and modality parameter set above. For HybridQA we keep modality='hybrid'. Other options are modality='table' and modality='passage'.

In [5]:
from primeqa.qg.models.qg_model import QGModel

qg_model = QGModel(model_name_or_path, modality=modality, lang='en')

/u/rbhat/.conda/envs/bhat/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# Loading Data

Here we load the data

In [6]:
from primeqa.qg.processors.data_loader import QGDataLoader

qgdl = QGDataLoader(
    tokenizer=qg_model.tokenizer,
    dataset_name=dataset_name,
    input_max_len=max_len,
    target_max_len=target_max_len
    )

train_dataset = qgdl.create("train[:100]")
valid_dataset = qgdl.create("validation[:50]")
print(train_dataset)
print(valid_dataset)

Reusing dataset hybrid_qa (/u/rbhat/.cache/huggingface/datasets/hybrid_qa/hybrid_qa/1.0.0/fabdc38783449dd6cb1acd25621af97b871e218fc3ab608191d492b408a93ab8)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:22<00:00,  1.43s/it]
Extracting chains: 77it [00:00, 392.64it/s]
Parameter 'function'=<function QGDataLoader.convert_to_features at 0x7f4dd0841820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/u/rbhat/.conda/envs/bhat/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Reusing dataset hybrid_qa (/u/rbhat/.cache/huggingface/datasets/hybrid_qa/hybrid_qa/1

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['question', 'input', 'input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'],
    num_rows: 75
})
Dataset({
    features: ['question', 'input', 'input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'],
    num_rows: 42
})


# Train using QGTrainer
Here we create a QG trainer with the training arguments defined above and use it to train on HybridQA training data (or any custom data following the same format)

In [7]:
import os
from primeqa.qg.trainers.qg_trainer import QGTrainer
from primeqa.qg.utils.data_collator import T2TDataCollator
from primeqa.qg.metrics.generation_metrics import rouge_metrics

compute_metrics = rouge_metrics(qg_model.tokenizer)

trainer = QGTrainer(
    model=qg_model.model,
    tokenizer = qg_model.tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    valid_dataset=valid_dataset,
    data_collator=T2TDataCollator(),
    compute_metrics=compute_metrics
    )

train_results = trainer.train()
trainer.save_model()
print(train_results.metrics)

/u/rbhat/.conda/envs/bhat/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 75
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 20


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.463604,13.918400,2.618900,11.397800,11.172200
2,No log,3.384258,13.007100,1.920900,10.423300,10.552000


***** Running Evaluation *****
  Num examples = 42
  Batch size = 32
***** Running Evaluation *****
  Num examples = 42
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /dccstor/cssblr/rbhat//models/
Configuration saved in /dccstor/cssblr/rbhat//models/config.json
Model weights saved in /dccstor/cssblr/rbhat//models/pytorch_model.bin
tokenizer config file saved in /dccstor/cssblr/rbhat//models/tokenizer_config.json
Special tokens file saved in /dccstor/cssblr/rbhat//models/special_tokens_map.json
Copy vocab file to /dccstor/cssblr/rbhat//models/spiece.model


{'train_runtime': 53.2622, 'train_samples_per_second': 2.816, 'train_steps_per_second': 0.376, 'total_flos': 7930183680000.0, 'train_loss': 3.827387237548828, 'epoch': 2.0}


## Evaluation

Here we evaluate the trained model on validation set

In [8]:
metrics = trainer.evaluate()
print(metrics)

***** Running Evaluation *****
  Num examples = 42
  Batch size = 32


{'eval_loss': 3.3842577934265137, 'eval_rouge1': 13.0071, 'eval_rouge2': 1.9209, 'eval_rougeL': 10.4233, 'eval_rougeLsum': 10.552, 'eval_runtime': 6.1628, 'eval_samples_per_second': 6.815, 'eval_steps_per_second': 0.325, 'epoch': 2.0}
